In [10]:
import csv
import time
from instagram_private_api import Client, ClientError

# Configura tus credenciales de Instagram
username = 'gsbg2101'
password = 'Lg030920.'

# Autenticación en Instagram
api = Client(username, password)

# Palabras clave de búsqueda en inglés y español
search_terms = ['Ropa', 'Moda', 'Estilo', 'Vestimenta', 'Marketing', 'Publicidad', 
                'Deportes', 'Ejercicio', 'Clothing', 'Fashion', 'Style', 
                'Advertising', 'Sports', 'Fitness', 'Tienda de deportes', 'Sport Store']

# Almacenar resultados
results = []

# Función para paginar los resultados de búsqueda
def search_and_collect(api, term, max_pages=10, results_per_page=50):
    next_max_id = ''
    for page in range(max_pages):
        try:
            # Realizar la búsqueda
            search_results = api.search_users(term, rank_token=api.generate_uuid(), max_id=next_max_id)
            users = search_results.get('users', [])
            if not users:
                break

            # Recoger resultados
            for user in users:
                profile_url = f"https://www.instagram.com/{user['username']}/"
                results.append({
                    'username': user['username'],
                    'full_name': user['full_name'],
                    'profile_pic_url': user['profile_pic_url'],
                    'user_id': user['pk'],
                    'profile_url': profile_url
                })

            # Actualizar el max_id para paginar
            next_max_id = search_results.get('next_max_id')
            if not next_max_id:
                break

            # Esperar entre las solicitudes para evitar ser bloqueado
            time.sleep(2)

        except ClientError as e:
            print(f"Error en la búsqueda para '{term}': {e}")
            break

# Buscar usuarios por cada término
for term in search_terms:
    search_and_collect(api, term)

# Guardar resultados en un archivo CSV
with open('instagram_profiles.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['username', 'full_name', 'image_formula', 'profile_pic_url', 'user_id', 'profile_url'])
    writer.writeheader()

    for index, result in enumerate(results, start=2):  # Comenzar desde la fila 2
        # Formula IMAGE sin comillas
        image_formula = f'=IMAGE(D{index}, 4, 50, 100)'
        result['image_formula'] = image_formula
        writer.writerow(result)

print("Archivo CSV creado con fórmulas IMAGE para Google Sheets")

Archivo CSV creado con fórmulas IMAGE para Google Sheets


In [ ]:
import csv
import time
from instagrapi import Client

# Configura tus credenciales de Instagram
username = 'gsbg2101'
password = 'Lg030920.'

# Autenticación en Instagram
cl = Client()
cl.login(username, password)

# Leer el archivo CSV de perfiles
with open('instagram_profiles.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    profiles = [row for row in reader]

# Enviar mensajes directos a cada usuario
for profile in profiles:
    username = profile['username']
    full_name = profile['full_name']
    user_id = int(profile['user_id'])  # Convertir a int si es necesario
    
    # Mensaje personalizado
    message = f"Hola {full_name},\n\nEspero que estés teniendo un gran día. ¡Saludos desde mi proyecto!"

    try:
        # Enviar mensaje directo usando instagrapi
        cl.direct_send(message, [user_id])
        print(f"Mensaje enviado a {username} ({full_name})")

        # Esperar antes de enviar el siguiente mensaje
        time.sleep(5)  # Cambiar a un tiempo mayor si es necesario

    except Exception as e:
        print(f"Error al enviar mensaje a {username}: {e}")

print("Mensajes enviados a todos los usuarios del CSV.")
